In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_excel('/content/drive/MyDrive/Capstone (Product)/Dataset/Resep Makanan/Dataset Cleaned/Dataset_Ikan.xlsx')
df = df.iloc[:40]
df


,Title,Ingridients
0,gurame saus padang,"[' gurame', 'bumbu saus', ' bawang putih ', ' ..."
1,ikan kembung bakar teflon,"[' ikan kembung', ' jeruk sate jeruk kunci', '..."
2,mujaer asam pedas manis,"[' ikan mujaer ', ' wortel', ' bawang bombai',..."
3,gurame asam manis,"[' ikan gurame', ' wortel ', 'bumbu', ' bawang..."
4,asam padeh ikan tuna,"[' ikan tuna', ' bawang merah', ' bawang putih..."
5,avocado tuna salad,"[' kaleng tuna chunks', 'telur rebus', ' kenta..."
6,lele asam manis,"[' ikan lele', ' air', ' kecap manis', ' marga..."
7,pempek kulit ikan tenggiri,"[' kulit ikan tenggiri', ' garam', ' penyedap ..."
8,tumis tongkol petai,"[' tongkol', ' cabe gendot', ' papan petai', '..."
9,ikan patin pedas manis,"[' ikan patin', ' bawang merah', ' bawang puti..."


In [5]:
#Fungsi untuk membersihkan teks
def clean_ingredients(text):
    text = re.sub(r'\d+|\s+|\-|\/', ' ', str(text))
    text =re.sub(r'''\d+|\s+|\-|\/|kg|gr|lembar|bahan|serbaguna|siung|ruas|jari|bungkus|buah|instan uk + campur tepung terigu|jdyg bsah. beli pasaraku beli rb
    cm|ekr|ne i cuci bersih ptng air jeruk nipis cuci lagi|ekor|ml|sdm|adonan basah|sdt|iris|sck|menggoreng|proteiin tinggi|bh|air merebus mie|kating|cuci bersih''', ' ', str(text))
    # Menghilangkan tanda kutip tunggal dan kurung siku
    text = re.sub(r"['\[\]]", '', text)
    # Menghilangkan beberapa stopwords
    stop_words = set(stopwords.words('indonesian'))
    stop_words.update(['secukupnya','bumbu','bumbu halus','kating',' - sesuaikan sesuaikan','bahan kuah'])
    words = text.split()
    # filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = [word.lower() for word in words if word.lower() not in stop_words]

     # Menghapus tanda koma pada akhir setiap kalimat
    cleaned_text = ' '.join(filtered_words)
    # cleaned_text = re.sub(r',\s*$', '', cleaned_text)

    return cleaned_text

In [6]:
# Membersihkan data pada kolom 'Ingredients'
bahan = df['Ingridients']
bahan = bahan.apply(clean_ingredients)

In [7]:
df['Bahan'] = bahan

In [8]:
judul = df['Title']


In [9]:
new_df = pd.DataFrame(df['Title'])
new_df['Bahan'] = bahan

In [10]:
# Menggabungkan dua kolom dengan tanda titik dua
new_df['Combined'] = new_df.apply(lambda row: f"{row['Title']}: {row['Bahan']}", axis=1)

# Menampilkan hasil
pd.DataFrame(new_df)


,Title,Bahan,Combined
0,gurame saus padang,"gurame, saus, bawang putih , bawang merah , ca...","gurame saus padang: gurame, saus, bawang putih..."
1,ikan kembung bakar teflon,"ikan kembung, jeruk sate jeruk kunci, garam ha...","ikan kembung bakar teflon: ikan kembung, jeruk..."
2,mujaer asam pedas manis,"ikan mujaer , wortel, bawang bombai, bawang pu...","mujaer asam pedas manis: ikan mujaer , wortel,..."
3,gurame asam manis,"ikan gurame, wortel , bumbu, bawang merah, baw...","gurame asam manis: ikan gurame, wortel , bumbu..."
4,asam padeh ikan tuna,"ikan tuna, bawang merah, bawang putih, kemiri,...","asam padeh ikan tuna: ikan tuna, bawang merah,..."
5,avocado tuna salad,"kaleng tuna chunks, telur rebus, kentang rebus...","avocado tuna salad: kaleng tuna chunks, telur ..."
6,lele asam manis,"ikan lele, air, kecap manis, margarine,minyak,...","lele asam manis: ikan lele, air, kecap manis, ..."
7,pempek kulit ikan tenggiri,"kulit ikan tenggiri, garam, penyedap kaldu jam...",pempek kulit ikan tenggiri: kulit ikan tenggir...
8,tumis tongkol petai,"tongkol, cabe gendot, papan petai, cabe rawit,...","tumis tongkol petai: tongkol, cabe gendot, pap..."
9,ikan patin pedas manis,"ikan patin, bawang merah, bawang putih, bawang...","ikan patin pedas manis: ikan patin, bawang mer..."


In [11]:
# Menyimpan DataFrame ke file Excel
# new_df.to_excel('Dataset_Fix.xlsx', index=False)

In [12]:
data = new_df['Combined']
pd.DataFrame(data)

,Combined
0,"gurame saus padang: gurame, saus, bawang putih..."
1,"ikan kembung bakar teflon: ikan kembung, jeruk..."
2,"mujaer asam pedas manis: ikan mujaer , wortel,..."
3,"gurame asam manis: ikan gurame, wortel , bumbu..."
4,"asam padeh ikan tuna: ikan tuna, bawang merah,..."
5,"avocado tuna salad: kaleng tuna chunks, telur ..."
6,"lele asam manis: ikan lele, air, kecap manis, ..."
7,pempek kulit ikan tenggiri: kulit ikan tenggir...
8,"tumis tongkol petai: tongkol, cabe gendot, pap..."
9,"ikan patin pedas manis: ikan patin, bawang mer..."


In [14]:

# Tokenisasi dataset
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index) + 1

# Sequencing data
input_sequences = []
for line in data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Pisahkan input dan output
X, y = input_sequences[:,:-1],input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Bangun model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 50, input_length=max_sequence_length-1),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(total_words, activation='softmax')
])
# model.add(Embedding(total_words, 50, input_length=max_sequence_length-1))
# model.add(LSTM(100))
# model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Latih model
model.fit(X, y, epochs=100, verbose=1)



Epoch 1/100
35/35 [==============================] - 3s 39ms/step - loss: 5.4235 - accuracy: 0.0664
Epoch 2/100
35/35 [==============================] - 1s 35ms/step - loss: 4.9171 - accuracy: 0.0728
Epoch 3/100
35/35 [==============================] - 1s 28ms/step - loss: 4.8550 - accuracy: 0.0728
Epoch 4/100
35/35 [==============================] - 1s 28ms/step - loss: 4.8182 - accuracy: 0.0728
Epoch 5/100
35/35 [==============================] - 1s 28ms/step - loss: 4.7718 - accuracy: 0.0764
Epoch 6/100
35/35 [==============================] - 1s 27ms/step - loss: 4.6989 - accuracy: 0.1010
Epoch 7/100
35/35 [==============================] - 1s 28ms/step - loss: 4.5960 - accuracy: 0.1092
Epoch 8/100
35/35 [==============================] - 1s 28ms/step - loss: 4.4807 - accuracy: 0.1374
Epoch 9/100
35/35 [==============================] - 1s 28ms/step - loss: 4.3495 - accuracy: 0.1756
Epoch 10/100
35/35 [==============================] - 1s 27ms/step - loss: 4.2344 - accuracy: 0.1747

#Model Generate Resep

In [16]:
# Fungsi generate resep
def generate_recipe(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probabilities = model.predict(token_list, verbose=0)

        predicted_index = np.argmax(predicted_probabilities)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word

    return seed_text

# Gunakan model untuk menghasilkan resep baru
nama_masakan = input(str('Masukan Nama Masakan: '))
generated_recipe = generate_recipe(nama_masakan, 10, model, max_sequence_length)
print(generated_recipe)

Masukan Nama Masakan: gurame asam manis
gurame asam manis manis: ikan gurame, wortel , bumbu, bawang merah, bawang putih,
